In [1]:
import pandas as pd
import numpy as np
import datetime
import plotly.graph_objects as go
from plotly.subplots import make_subplots

#import sdk library from parent directory
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
colors = {
    'grid': "rgba(255,255,255,0.2)",
    'graph_font': "#f0f8ff",
    'plot_area': "rgba(50,50,50,0.2)",
    'plot_background': "rgba(50,50,50,0.5)",
    'active_link': 'rgba(200,200,200,0.7)',
    'disabled_link': 'rgba(200,100,100,0.7)',
}

In [3]:
import pandas as pd
import numpy as np
import dash
import datetime
from dash import dcc, html, dash_table, callback_context
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc

from sdk import SedmaxHeader, Sedmax, ElectricalArchive, EventJournal, RTDArchive
from uptime_report.graphs import out_time_scatter, out_table


energy = 'ea' #'er' or 'ea'


s = Sedmax('https://demo.sedmax.ru')

username = 'demo' #os.environ['SEDMAX_USERNAME']
password = 'demo' #os.environ['SEDMAX_PASSWORD']
s.login(username, password)

el = ElectricalArchive(s)
df = el.get_data(["dev-101_"+energy+"_imp","dev-109_"+energy+"_imp","dev-110_"+energy+"_imp",
                "dev-111_"+energy+"_imp","dev-112_"+energy+"_imp","dev-114_"+energy+"_imp",
                "dev-108_"+energy+"_imp","dev-102_"+energy+"_imp","dev-103_"+energy+"_imp",
                "dev-104_"+energy+"_imp","dev-105_"+energy+"_imp","dev-106_"+energy+"_imp",
                "dev-107_"+energy+"_imp","dev-202_"+energy+"_imp","dev-201_"+energy+"_imp",
                "dev-113_"+energy+"_imp","dev-116_"+energy+"_imp","dev-117_"+energy+"_imp"], 
                 ['30min'], '2022-03-21','2022-03-21')

tree = el.devices_tree()
parents = tree.parent.unique()

names = tree.name.values.tolist()
names.append('object-0')
map_dict = tree.id.values.tolist()
map_dict.append('object-0')
map_dict = dict([(v,k) for (k,v) in enumerate(map_dict)])

power_data = df.sum()
power_data.index = [x.split('_')[0].replace('dev', 'device') for x in power_data.index]
power_data['device-110'] = power_data['device-110']/1000

power_data.index = power_data.index.map(map_dict)
tree.id = tree.id.map(map_dict)

records = []
for par in parents:

    children = tree[tree.parent == par].id.to_list()
    values = []
    for device in children:
        if device in power_data.index:
            values.append(power_data[device])
        else:
            values.append(0)

    Dict = {'name': map_dict[par],
          'children': tree[tree.parent == par].id.to_list(),
          'values': values
          }
    records.append(Dict)

    
#manually determined records    

records = [
        {'dev': 23, 'children': [8]},
        {'dev': 8, 'children': [1]},
        {'dev': 1, 'children': [15, 16, 17, 18, 25]},
        {'dev': 24, 'children': [9]},
        {'dev': 9, 'children': [2]},
        {'dev': 2, 'children': [10, 11, 12, 13, 14]},
        {'dev': 12, 'children': [20]},
        {'dev': 18, 'children': [19]},
        {'dev': 14, 'children': [19]},
        {'dev': 19, 'children': [21]},
        {'dev': 20, 'children': [22]},
    ]


def flatten(t):
    return [item for sublist in t for item in sublist]

source = []
target = []
value = []
link_colors = []

for record in records:
    children = record['children']
    device = record['dev']
    values = []

    if len(children) <= 1:
        if device in power_data.index:
            if power_data[device] != 0:
                values.append(power_data[device])
                link_colors.append(colors['active_link'])
            else:
                values.append(0.01)
                link_colors.append(colors['disabled_link'])
        else:
            values.append(0.01)
            link_colors.append(colors['disabled_link'])
    else:
        for child in children:
            if child in power_data.index:
                if power_data[child] != 0:
                    values.append(power_data[child])
                    link_colors.append(colors['active_link'])
                else:
                    values.append(0.01)
                    link_colors.append(colors['disabled_link'])
            else:
                values.append(0.01)
                link_colors.append(colors['disabled_link'])

    node = [record['dev']] * len(children)

    source.append(node)
    target.append(children)
    value.append(values)

source = flatten(source)
target = flatten(target)
value = flatten(value)
labels = names

print(len(source), len(target), len(value), len(labels))

Exception: Status code: 503, message: Forbidden

In [9]:
s.channels_tree()

AttributeError: 'Sedmax' object has no attribute 'channels_tree'

In [63]:
colors = {
    'grid': "rgba(255,255,255,0.2)",
    'graph_font': "#f0f8ff",
    'plot_area': "rgba(50,50,50,0.2)",
    'plot_background': "rgba(50,50,50,0.5)",
    'active_link': 'rgba(200,200,200,0.7)',
    'disabled_link': 'rgba(200,100,100,0.7)',
}

In [156]:
fig = go.Figure(data=[go.Sankey(
    valuesuffix = " кВтч",
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      label = names,
      #color = "blue"
    ),
    link = dict(
      source = source, # indices correspond to labels, eg A1, A2, A1, B1, ...
      target = target,
      value = value,
      #label = names  
  ))])
#fig.update_layout(title_text="Basic Sankey Diagram", font_size=10)

fig.update_layout(height=700,
                      font_color=colors['graph_font'],
                      title_text='Basic Sankey Diagram',
                      plot_bgcolor=colors['plot_area'],
                      paper_bgcolor=colors['plot_background'],
                      margin=dict(l=20, r=20, b=30, t=80, pad=1),
                      )

fig.show()


In [5]:
tree = el.devices_tree()
tree

,id,name,parent,type
0,object-1,Офис,object-0,1
1,object-2,ВРУ-3.1,object-1,1
2,object-3,ВРУ-3.2,object-1,1
3,object-4,ВРУ 2-го этажа,object-1,1
4,object-5,ВРУ 1-го этажа,object-1,1
5,object-21,Шкаф АВР,object-1,1
6,object-23,Серверная,object-1,1
7,device-0,Модуль дорасчетов,object-0,2
8,device-101,Ввод-1 (контр.),object-2,2
9,device-102,Ввод-2 (контр.),object-3,2


In [7]:
power_data = df.sum()
power_data.index = [x.split('_')[0].replace('dev', 'device') for x in power_data.index]
#power_data['device-110'] = power_data['device-110']/2000 
#power_data['device-117'] = power_data['device-117']/10
#power_data['device-109'] = power_data['device-109']/100

power_data.index = power_data.index.map(map_dict)
power_data

8      34.5820
9     236.6190
10    102.4345
11     46.5925
12    140.2055
13      0.0000
14      0.0000
25      0.3840
15      0.3540
16     33.3620
17      0.0000
18      0.0000
19     23.2995
20     18.3435
21     23.2995
22     23.2995
23     34.6230
24    236.5070
dtype: float64

In [8]:
records = [

{'dev':23, 'children': [8]},
{'dev':8, 'children': [1]},   
 {'dev':1, 'children': [15,16,17,18,25]},
 {'dev':24, 'children': [9]},
{'dev':9, 'children': [2]},  
 {'dev':2, 'children': [10,11,12,13,14]},
 {'dev':12, 'children': [20]},
 {'dev':18, 'children': [19]},
 {'dev':14, 'children': [19]},
 {'dev':19, 'children': [21]},        
 {'dev':20, 'children': [22]},   
]



In [9]:
records = [

{'dev':23, 'children': [8]},
 {'dev':8, 'children': [15,16,17,18,25]},
 {'dev':24, 'children': [9]},
 {'dev':9, 'children': [10,11,12,13,14]},
 {'dev':12, 'children': [20]},
 {'dev':18, 'children': [19]},
 {'dev':14, 'children': [19]},
 {'dev':19, 'children': [21]},        
 {'dev':20, 'children': [22]},   
]



In [10]:
source = []
target = []
value = []
link_colors = []

for record in records:
    children = record['children']
    device = record['dev']
    values = []

    if len(children) <= 1:
        if device in power_data.index:
            if power_data[device] != 0:
                values.append(power_data[device])
                link_colors.append(colors['active_link'])
            else:
                values.append(0.1)
                link_colors.append(colors['disabled_link'])
        else:
            values.append(0.1)
            link_colors.append(colors['disabled_link'])
    else:
        for child in children:
            if child in power_data.index:
                if power_data[child] != 0:
                    values.append(power_data[child])
                    link_colors.append(colors['active_link'])
                else:
                    values.append(0.1)
                    link_colors.append(colors['disabled_link'])
            else:
                values.append(0.1)
                link_colors.append(colors['disabled_link'])

    node = [record['dev']] * len(children)

    source.append(node)
    target.append(children)
    value.append(values)

source = flatten(source)
target = flatten(target)
value = flatten(value)

In [11]:
value

[34.623,
 0.354,
 33.362,
 0.1,
 0.1,
 0.384,
 236.507,
 102.4345,
 46.5925,
 140.2055,
 0.1,
 0.1,
 140.2055,
 0.1,
 0.1,
 23.2995,
 18.3435]

In [12]:
names = tree.name.copy()
names[8] = 'ВРУ-3.1'
names[9] = 'ВРУ-3.2'
names = names.values


In [13]:
fig = go.Figure(data=[go.Sankey(
    valuesuffix = " кВтч",
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      label = names,
      #color = "blue"
    ),
    link = dict(
      source = source, # indices correspond to labels, eg A1, A2, A1, B1, ...
      target = target,
      value = value,
      color = link_colors
      #label = names  
  ))])

fig.update_layout(height=500,
                      #font_color=colors['graph_font'],
                      title_text='Basic Sankey Diagram',
                      #plot_bgcolor=colors['plot_area'],
                      #paper_bgcolor=colors['plot_background'],
                      margin=dict(l=20, r=20, b=30, t=80, pad=1),
                      )

fig.show()


In [22]:
tree

,id,name,parent,type
0,object-1,Офис,object-0,1
1,object-2,ВРУ-3.1,object-1,1
2,object-3,ВРУ-3.2,object-1,1
3,object-4,ВРУ 2-го этажа,object-1,1
4,object-5,ВРУ 1-го этажа,object-1,1
5,object-21,Шкаф АВР,object-1,1
6,object-23,Серверная,object-1,1
7,device-0,Модуль дорасчетов,object-0,2
8,device-101,Ввод-1 (контр.),object-2,2
9,device-102,Ввод-2 (контр.),object-3,2


In [14]:
power_data.to_frame().merge(tree, left_index=True, right_index=True, how='outer')

,0,id,name,parent,type
0,NaN,object-1,Офис,object-0,1
1,NaN,object-2,ВРУ-3.1,object-1,1
2,NaN,object-3,ВРУ-3.2,object-1,1
3,NaN,object-4,ВРУ 2-го этажа,object-1,1
4,NaN,object-5,ВРУ 1-го этажа,object-1,1
5,NaN,object-21,Шкаф АВР,object-1,1
6,NaN,object-23,Серверная,object-1,1
7,NaN,device-0,Модуль дорасчетов,object-0,2
8,34.5820,device-101,Ввод-1 (контр.),object-2,2
9,236.6190,device-102,Ввод-2 (контр.),object-3,2


In [53]:
df

,dev-101_ea_imp,dev-102_ea_imp,dev-103_ea_imp,dev-104_ea_imp,dev-105_ea_imp,dev-106_ea_imp,dev-107_ea_imp,dev-108_ea_imp,dev-109_ea_imp,dev-110_ea_imp,dev-111_ea_imp,dev-112_ea_imp,dev-113_ea_imp,dev-114_ea_imp,dev-116_ea_imp,dev-117_ea_imp,dev-201_ea_imp,dev-202_ea_imp
dt,,,,,,,,,,,,,,,,,,
2021-12-19 00:00:00,0.7155,4.5875,2.1345,0.8165,2.9060,0.0,0.0,0.008,0.0075,0.6915,0.0,0.0,0.5030,0.3950,0.5030,0.5030,0.6845,4.7890
2021-12-19 00:30:00,0.7335,4.7980,2.1345,0.8520,1.8895,0.0,0.0,0.008,0.0075,0.7065,0.0,0.0,0.4945,0.3870,0.4945,0.4945,0.7705,4.7365
2021-12-19 01:00:00,0.7420,4.7575,2.1345,0.8220,1.8880,0.0,NaN,0.008,0.0075,0.7165,0.0,0.0,0.4895,0.3835,0.4895,0.4895,0.7840,4.6955
2021-12-19 01:30:00,0.7510,4.8000,2.1345,0.8720,1.8820,0.0,0.0,0.008,0.0075,0.7265,0.0,0.0,0.4890,0.3825,0.4890,0.4890,0.7245,5.1035
2021-12-19 02:00:00,0.7510,4.7640,2.1345,0.8330,1.8825,0.0,0.0,0.008,0.0075,0.7255,0.0,0.0,0.4895,0.3830,0.4895,0.4895,0.7400,4.9485
2021-12-19 02:30:00,0.6885,4.7415,2.1345,0.8105,1.8835,0.0,0.0,0.008,0.0075,0.6640,0.0,0.0,0.4855,0.3800,0.4855,0.4855,0.6765,4.7870
2021-12-19 03:00:00,0.7250,4.7695,2.1345,0.8295,1.8960,0.0,0.0,0.008,0.0075,0.7000,0.0,0.0,0.4890,0.3835,0.4890,0.4890,0.6745,4.6355
2021-12-19 03:30:00,0.7370,4.7955,2.1345,0.8585,1.8915,0.0,0.0,0.008,0.0075,0.7110,0.0,0.0,0.4865,0.3810,0.4865,0.4865,0.7850,4.7800
2021-12-19 04:00:00,0.7460,4.7755,2.1345,0.8445,1.8855,0.0,0.0,0.008,0.0075,0.7215,0.0,0.0,0.4890,0.3830,0.4890,0.4890,0.7185,4.8345


In [54]:
tree1 = el.devices_tree()
tree1

,id,name,parent,type
0,object-1,Офис,object-0,1
1,object-2,ВРУ-3.1,object-1,1
2,object-3,ВРУ-3.2,object-1,1
3,object-4,ВРУ 2-го этажа,object-1,1
4,object-5,ВРУ 1-го этажа,object-1,1
5,object-21,Шкаф АВР,object-1,1
6,object-23,Серверная,object-1,1
7,device-0,Модуль дорасчетов,object-0,2
8,device-101,Ввод-1 (контр.),object-2,2
9,device-102,Ввод-2 (контр.),object-3,2


In [98]:
el.get_data(["dev-117_ea_imp"], ['30min'], '2021-12-19','2021-12-19')

,dev-117_ea_imp
dt,
2021-12-19 00:00:00,0.5030
2021-12-19 00:30:00,0.4945
2021-12-19 01:00:00,0.4895
2021-12-19 01:30:00,0.4890
2021-12-19 02:00:00,0.4895
2021-12-19 02:30:00,0.4855
2021-12-19 03:00:00,0.4890
2021-12-19 03:30:00,0.4865
2021-12-19 04:00:00,0.4890


In [55]:
sorted(["dev-101_ea_imp","dev-109_ea_imp","dev-110_ea_imp",
                "dev-111_ea_imp","dev-112_ea_imp","dev-114_ea_imp",
                "dev-108_ea_imp","dev-102_ea_imp","dev-103_ea_imp",
                "dev-104_ea_imp","dev-105_ea_imp","dev-106_ea_imp",
                "dev-107_ea_imp","dev-202_ea_imp","dev-201_ea_imp",
                "dev-113_ea_imp","dev-116_ea_imp","dev-117_ea_imp"])

['dev-101_ea_imp',
 'dev-102_ea_imp',
 'dev-103_ea_imp',
 'dev-104_ea_imp',
 'dev-105_ea_imp',
 'dev-106_ea_imp',
 'dev-107_ea_imp',
 'dev-108_ea_imp',
 'dev-109_ea_imp',
 'dev-110_ea_imp',
 'dev-111_ea_imp',
 'dev-112_ea_imp',
 'dev-113_ea_imp',
 'dev-114_ea_imp',
 'dev-116_ea_imp',
 'dev-117_ea_imp',
 'dev-201_ea_imp',
 'dev-202_ea_imp']

In [48]:
class SomeClass(object):
    def __init__(self, n):
        self.list = range(0, n)
    
    def get_list(self):
        return self._list
    @property
    def list(self):
        return self._list
    @list.setter
    def list(self, val):
        self._list = val
        self._listsquare = [x**2 for x in self._list ]

    @property
    def listsquare(self):
        return self._listsquare
    @listsquare.setter
    def listsquare(self, val):
        self.list = [int(pow(x, 0.5)) for x in val]
        
        
class ChildClass(object):
        def __init__(self, s):
            self.atrib = s.list
            self.parent = s
        def get(self):
            return self.parent.get_list()
            
        


In [54]:
a = SomeClass(3)

In [55]:
a.list

range(0, 3)

In [61]:
a.list = range(0, 5)

In [56]:
b = ChildClass(a)

In [65]:
b.atrib

range(0, 3)

In [66]:
b.get()

range(0, 5)

In [64]:
a.get_list()

range(0, 5)

### Диаграмма с активной мощностью

In [142]:
import requests

request = {"nodes": ['obj-1']}
url = s.host + '/sedmax/devices_configurator/ti/list'

r = requests.post(
            url,
            json=request,
            cookies={'jwt': s.token}
        )


In [143]:
data = pd.DataFrame(r.json()['ti'])

In [144]:
data['common-device'] = [x[-1] for x in data['path'].str.split('/')]
data

,tiId,name,path,deviceName,protocol,address,telecontrolled,unit,category,code,active,common-device
0,1,Свободное место на диске,Офис/Диагностируемое оборудование/Сервер SEDMAX,Сервер SEDMAX,SNMP v.1,1.3.6.1.4.1.2021.9.1.7.1,False,Гб,rtd,par-1,True,Сервер SEDMAX
1,2,Свободная оперативная память,Офис/Диагностируемое оборудование/Сервер SEDMAX,Сервер SEDMAX,SNMP v.1,.1.3.6.1.4.1.2021.4.11.0,False,Гб,rtd,par-2,True,Сервер SEDMAX
2,3,Средняя нагрузка на ЦПУ (15 минут),Офис/Диагностируемое оборудование/Сервер SEDMAX,Сервер SEDMAX,SNMP v.1,.1.3.6.1.4.1.2021.10.1.3.3,False,%,rtd,par-3,True,Сервер SEDMAX
3,1001,Напряжение фазы А,Офис/ВРУ-3.1/Ввод-1 (контр.),Ввод-1 (контр.),Modbus (TCP),0,False,В,rtd,par-1001,True,Ввод-1 (контр.)
4,1002,Напряжение фазы В,Офис/ВРУ-3.1/Ввод-1 (контр.),Ввод-1 (контр.),Modbus (TCP),2,False,В,rtd,par-1002,True,Ввод-1 (контр.)
...,...,...,...,...,...,...,...,...,...,...,...,...
1382,40011110,Напряжение,Офис/Серверная/Шкаф серверный №2 ввод-2,Шкаф серверный №2 ввод-2,Modbus (TCP),38,False,В,rtd,par-40011110,True,Шкаф серверный №2 ввод-2
1383,40011111,Энергия активная прием,Офис/Серверная/Шкаф серверный №2 ввод-2,Шкаф серверный №2 ввод-2,Modbus (TCP),26,False,кВт*ч,rtd,par-40011111,True,Шкаф серверный №2 ввод-2
1384,40011112,Энергия активная отдача,Офис/Серверная/Шкаф серверный №2 ввод-2,Шкаф серверный №2 ввод-2,Modbus (TCP),28,False,кВт*ч,rtd,par-40011112,True,Шкаф серверный №2 ввод-2
1385,40011113,Энергия реактивная прием,Офис/Серверная/Шкаф серверный №2 ввод-2,Шкаф серверный №2 ввод-2,Modbus (TCP),30,False,кВАр*ч,rtd,par-40011113,True,Шкаф серверный №2 ввод-2


In [145]:
data_dev = data[data['common-device'].isin(tree.name)]
data_dev

,tiId,name,path,deviceName,protocol,address,telecontrolled,unit,category,code,active,common-device
3,1001,Напряжение фазы А,Офис/ВРУ-3.1/Ввод-1 (контр.),Ввод-1 (контр.),Modbus (TCP),0,False,В,rtd,par-1001,True,Ввод-1 (контр.)
4,1002,Напряжение фазы В,Офис/ВРУ-3.1/Ввод-1 (контр.),Ввод-1 (контр.),Modbus (TCP),2,False,В,rtd,par-1002,True,Ввод-1 (контр.)
5,1003,Напряжение фазы С,Офис/ВРУ-3.1/Ввод-1 (контр.),Ввод-1 (контр.),Modbus (TCP),4,False,В,rtd,par-1003,True,Ввод-1 (контр.)
6,1004,Напряжение фазное среднее,Офис/ВРУ-3.1/Ввод-1 (контр.),Ввод-1 (контр.),Modbus (TCP),6,False,В,rtd,par-1004,True,Ввод-1 (контр.)
7,1005,Напряжение линейное A-B,Офис/ВРУ-3.1/Ввод-1 (контр.),Ввод-1 (контр.),Modbus (TCP),8,False,В,rtd,par-1005,True,Ввод-1 (контр.)
...,...,...,...,...,...,...,...,...,...,...,...,...
1382,40011110,Напряжение,Офис/Серверная/Шкаф серверный №2 ввод-2,Шкаф серверный №2 ввод-2,Modbus (TCP),38,False,В,rtd,par-40011110,True,Шкаф серверный №2 ввод-2
1383,40011111,Энергия активная прием,Офис/Серверная/Шкаф серверный №2 ввод-2,Шкаф серверный №2 ввод-2,Modbus (TCP),26,False,кВт*ч,rtd,par-40011111,True,Шкаф серверный №2 ввод-2
1384,40011112,Энергия активная отдача,Офис/Серверная/Шкаф серверный №2 ввод-2,Шкаф серверный №2 ввод-2,Modbus (TCP),28,False,кВт*ч,rtd,par-40011112,True,Шкаф серверный №2 ввод-2
1385,40011113,Энергия реактивная прием,Офис/Серверная/Шкаф серверный №2 ввод-2,Шкаф серверный №2 ввод-2,Modbus (TCP),30,False,кВАр*ч,rtd,par-40011113,True,Шкаф серверный №2 ввод-2


In [146]:
data_dev = data_dev[data_dev.name == 'Активная мощность суммарная'].merge(tree, left_on = 'common-device', right_on = 'name')
data_dev

,tiId,name_x,path,deviceName,protocol,address,telecontrolled,unit,category,code,active,common-device,id,name_y,parent,type
0,1016,Активная мощность суммарная,Офис/ВРУ-3.1/Ввод-1 (контр.),Ввод-1 (контр.),Modbus (TCP),30,False,кВт,rtd,par-1016,True,Ввод-1 (контр.),8,Ввод-1 (контр.),object-2,2
1,2016,Активная мощность суммарная,Офис/ВРУ-3.2/Ввод-2 (контр.),Ввод-2 (контр.),IEC 60870-5-104,"16 (13,36)",False,кВт,rtd,par-2016,True,Ввод-2 (контр.),9,Ввод-2 (контр.),object-3,2
2,3016,Активная мощность суммарная,Офис/ВРУ-3.2/ЩВК 3.3 (ВРУ 3.2),ЩВК 3.3 (ВРУ 3.2),Modbus (TCP),30,False,кВт,rtd,par-3016,True,ЩВК 3.3 (ВРУ 3.2),10,ЩВК 3.3 (ВРУ 3.2),object-3,2
3,4016,Активная мощность суммарная,Офис/ВРУ-3.2/ЩР 3.2 (ВРУ 3.2),ЩР 3.2 (ВРУ 3.2),Modbus (TCP),30,False,кВт,rtd,par-4016,True,ЩР 3.2 (ВРУ 3.2),11,ЩР 3.2 (ВРУ 3.2),object-3,2
4,5016,Активная мощность суммарная,Офис/ВРУ-3.2/ЩРС 2 (ВРУ 3.2),ЩРС 2 (ВРУ 3.2),IEC 60870-5-104,"16 (13,36)",False,кВт,rtd,par-5016,True,ЩРС 2 (ВРУ 3.2),12,ЩРС 2 (ВРУ 3.2),object-3,2
5,6016,Активная мощность суммарная,Офис/ВРУ-3.2/Резерв (ВРУ 3.2),Резерв (ВРУ 3.2),IEC 60870-5-104,"16 (13,36)",False,кВт,rtd,par-6016,True,Резерв (ВРУ 3.2),13,Резерв (ВРУ 3.2),object-3,2
6,7016,Активная мощность суммарная,Офис/ВРУ-3.2/ЩАВР (ВРУ 3.2),ЩАВР (ВРУ 3.2),Modbus (TCP),30,False,кВт,rtd,par-7016,True,ЩАВР (ВРУ 3.2),14,ЩАВР (ВРУ 3.2),object-3,2
7,8007,Активная мощность суммарная,Офис/ВРУ-3.1/ЩВК 3.1 (ВРУ 3.1),ЩВК 3.1 (ВРУ 3.1),Modbus (TCP),12,False,кВт,rtd,par-8007,True,ЩВК 3.1 (ВРУ 3.1),25,ЩВК 3.1 (ВРУ 3.1),object-2,2
8,8107,Активная мощность суммарная,Офис/ВРУ-3.1/ЩВК 3.2 (ВРУ 3.1),ЩВК 3.2 (ВРУ 3.1),Modbus (TCP),12,False,кВт,rtd,par-8107,True,ЩВК 3.2 (ВРУ 3.1),15,ЩВК 3.2 (ВРУ 3.1),object-2,2
9,8207,Активная мощность суммарная,Офис/ВРУ-3.1/ЩР 3.1 (ВРУ 3.1),ЩР 3.1 (ВРУ 3.1),Modbus (TCP),12,False,кВт,rtd,par-8207,True,ЩР 3.1 (ВРУ 3.1),16,ЩР 3.1 (ВРУ 3.1),object-2,2


In [147]:
rtd = RTDArchive(s)

In [148]:
tags = data_dev.code.to_list()

powe = rtd.get_data(tags, len(tags), '2022-03-21 00:00:00','2022-03-21 23:59:59')

In [149]:
vals = pd.DataFrame(powe[[x for x in powe.columns if x.split('-')[-1] == 'value']].mean()).reset_index()
vals['code'] = [x[0] for x in vals['index'].str.split('-value')]
vals

,index,0,code
0,par-10016-value,4.968648,par-10016
1,par-1016-value,4.949575,par-1016
2,par-11016-value,12.579769,par-11016
3,par-2016-value,12.494300,par-2016
4,par-3016-value,0.716599,par-3016
5,par-4016-value,4.291100,par-4016
6,par-5016-value,5.891100,par-5016
7,par-6016-value,0.000000,par-6016
8,par-7016-value,4.602560,par-7016
9,par-8007-value,0.606533,par-8007


In [150]:
import pandas as pd
import numpy as np
import dash
import datetime
from dash import dcc, html, dash_table, callback_context
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc

from sdk import SedmaxHeader, Sedmax, ElectricalArchive, EventJournal, RTDArchive
from uptime_report.graphs import out_time_scatter, out_table


energy = 'ea' #'er' or 'ea'


s = Sedmax('https://demo.sedmax.ru')

username = 'demo' #os.environ['SEDMAX_USERNAME']
password = 'demo' #os.environ['SEDMAX_PASSWORD']
s.login(username, password)

el = ElectricalArchive(s)

tree = el.devices_tree()
parents = tree.parent.unique()

names = tree.name.values.tolist()
names.append('object-0')
map_dict = tree.id.values.tolist()
map_dict.append('object-0')
map_dict = dict([(v,k) for (k,v) in enumerate(map_dict)])

tree.id = tree.id.map(map_dict)

power_data = data_dev.merge(vals)[['id', 0]].set_index('id')[0]

records = []
for par in parents:

    children = tree[tree.parent == par].id.to_list()
    values = []
    for device in children:
        if device in power_data.index:
            values.append(power_data[device])
        else:
            values.append(0)

    Dict = {'name': map_dict[par],
          'children': tree[tree.parent == par].id.to_list(),
          'values': values
          }
    records.append(Dict)

    
#manually determined records    

records = [
        {'dev': 23, 'children': [8]},
        {'dev': 8, 'children': [1]},
        {'dev': 1, 'children': [15, 16, 17, 18, 25]},
        {'dev': 24, 'children': [9]},
        {'dev': 9, 'children': [2]},
        {'dev': 2, 'children': [10, 11, 12, 13, 14]},
        {'dev': 12, 'children': [20]},
        {'dev': 18, 'children': [19]},
        {'dev': 14, 'children': [19]},
        {'dev': 19, 'children': [21]},
        {'dev': 20, 'children': [22]},
    ]


def flatten(t):
    return [item for sublist in t for item in sublist]

source = []
target = []
value = []
link_colors = []

for record in records:
    children = record['children']
    device = record['dev']
    values = []

    if len(children) <= 1:
        if device in power_data.index:
            if power_data[device] != 0:
                values.append(power_data[device])
                link_colors.append(colors['active_link'])
            else:
                values.append(0.01)
                link_colors.append(colors['disabled_link'])
        else:
            values.append(0.01)
            link_colors.append(colors['disabled_link'])
    else:
        for child in children:
            if child in power_data.index:
                if power_data[child] != 0:
                    values.append(power_data[child])
                    link_colors.append(colors['active_link'])
                else:
                    values.append(0.01)
                    link_colors.append(colors['disabled_link'])
            else:
                values.append(0.01)
                link_colors.append(colors['disabled_link'])

    node = [record['dev']] * len(children)

    source.append(node)
    target.append(children)
    value.append(values)

source = flatten(source)
target = flatten(target)
value = flatten(value)
labels = names

print(len(source), len(target), len(value), len(labels))

19 19 19 27


In [151]:
fig = go.Figure(data=[go.Sankey(
    valuesuffix = " кВтч",
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      label = names,
      #color = "blue"
    ),
    link = dict(
      source = source, # indices correspond to labels, eg A1, A2, A1, B1, ...
      target = target,
      value = value,
      #label = names  
  ))])
#fig.update_layout(title_text="Basic Sankey Diagram", font_size=10)

fig.update_layout(height=700,
                      font_color=colors['graph_font'],
                      title_text='Basic Sankey Diagram',
                      plot_bgcolor=colors['plot_area'],
                      paper_bgcolor=colors['plot_background'],
                      margin=dict(l=20, r=20, b=30, t=80, pad=1),
                      )

fig.show()


AttributeError: 'RTDArchive' object has no attribute 'devices_tree'